In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
df_items=pd.read_csv('df_items_limpio.csv')
df_reviews=pd.read_csv('df_reviews_limpio.csv')
df_games=pd.read_csv('df_games_limpio.csv')

# USERDATA csv

In [10]:
df_reviews

,user_id,user_url,item_id,recommend,review,date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,1,2,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,1,2,2011-07-15
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,1,2,2011-04-21
3,js41637,http://steamcommunity.com/id/js41637,251610,1,2,2014-06-24
4,js41637,http://steamcommunity.com/id/js41637,227300,1,0,2013-09-08
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,1,2,NaN
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,1,2,NaN
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,1,2,NaN
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,730,1,2,NaN


In [4]:
df_items_count = df_items.groupby('user_id')['items_count'].mean().reset_index()
df_items_count.sort_values(by=items)

,user_id,items_count
0,--000--,58.0
1,--ace--,44.0


In [11]:
df_items

,user_id,user_url,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,10,counter-strike,6,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,20,team fortress classic,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,30,day of defeat,7,0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,40,deathmatch classic,0,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,50,half-life: opposing force,0,0
...,...,...,...,...,...,...,...,...
5094100,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7,76561198329548331,346330,brainbread 2,0,0
5094101,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7,76561198329548331,373330,all is dust,0,0
5094102,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7,76561198329548331,388490,one way to die: steam edition,3,3
5094103,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7,76561198329548331,521570,you have 10 seconds 2,4,4


In [6]:
result=df_reviews[['user_id','recommend']]
df_user_recommend_ratio = result.groupby('user_id')['recommend'].mean().round(2).reset_index()
df_user_recommend_ratio.head(2)

,user_id,recommend
0,--000--,1.0
1,--ace--,1.0


In [7]:
df_usuarios_items = df_items.groupby('user_id')['item_id'].apply(list).reset_index()

In [8]:
df_usuarios_items.head(2)

,user_id,item_id
0,--000--,"[4000, 1250, 35420, 550, 223530, 43110, 105600..."
1,--ace--,"[4000, 23490, 40800, 620, 113200, 205790, 730,..."


In [9]:
df_usuarios_items.to_csv('itemsforuser.csv', index=False)

In [126]:
# Crear un diccionario que mapea el id del juego a su precio
game_id_to_price = df_games.set_index('id')['price'].to_dict() #importantisimaa
# Calcular el total gastado para cada usuario
df_usuarios_items['total_spent'] = df_usuarios_items['item_id'].apply(lambda items: sum(game_id_to_price.get(item, 0) for item in items))

#game_id_to_price.get(item, 0) for item in items obtiene el precio de cada item por cada item en item_id


In [127]:
df_gasto=df_usuarios_items.drop('item_id', axis=1)
df_gasto.head(2)

,user_id,total_spent
0,--000--,402.77
1,--ace--,166.82


In [128]:
lista=[df_gasto, df_user_recommend_ratio]
temp=df_items_count
for i in range(len(lista)):
    df_user_data= pd.merge(temp,lista[i], on='user_id', how='outer')
    temp=df_user_data
        

In [129]:
df_user_data.head(2)
df_user_data.loc[:, 'recommend'] = df_user_data['recommend'] * 100
df_user_data.rename(columns={'recommend': 'recommend_ratio'})

,user_id,items_count,total_spent,recommend_ratio
0,--000--,58.0,402.77,100.0
1,--ace--,44.0,166.82,100.0
2,--ionex--,23.0,109.92,100.0
3,-2SV-vuLB-Kg,68.0,437.49,100.0
4,-404PageNotFound-,149.0,1514.31,NaN
...,...,...,...,...
73752,zixwot,NaN,NaN,67.0
73753,zombifiedddd,NaN,NaN,100.0
73754,zoozles,NaN,NaN,100.0
73755,zraicis,NaN,NaN,100.0


In [130]:
df_user_data.to_csv('df_user_data.csv', index=False)

# creacion de funcion userdata

In [15]:
df_user_data=pd.read_csv('df_user_data.csv')

In [32]:
def userdata(user_id):
    df = df_user_data.loc[df_user_data['user_id'] == f'{user_id}']
    #df.loc[:, 'recommend'] = df['recommend']
    df=df.rename(columns={'recommend': 'porcentaje de recomendacion',
                 'items_count': 'total items', 'total_spent':'total gastado'})

    return df
    

In [30]:
xd=userdata('--000--')
xd

,user_id,items_count,total_spent,recoomend_ratio
0,--000--,58.0,402.77,100.0


# COUNTREVIEWS csv

In [139]:
df_reviews.head(2)

,user_id,user_url,item_id,recommend,review,date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,1,2,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,1,2,2011-07-15


In [145]:
df_count_reviews=df_reviews[['user_id','date']]
df_count_reviews = df_count_reviews.merge(df_user_recommend_ratio, on='user_id', how='left')
df_count_reviews=df_count_reviews.dropna(subset='date')
df_count_reviews.head(2)

,user_id,date,recommend
0,76561197970982479,2011-11-05,1.0
1,76561197970982479,2011-07-15,1.0


In [146]:
df_count_reviews.to_csv('df_count_reviews.csv', index=False)

# creacion de la funcion countreviews

In [12]:
df_count_reviews=pd.read_csv('df_count_reviews.csv')

In [13]:
def countreviews(fecha_inicio,fecha_fin):
    df_count_reviews['date'] = pd.to_datetime(df_count_reviews['date'])
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)

    # Filtrar el DataFrame para el rango de fechas deseado
    df_rango = df_count_reviews[(df_count_reviews['date'] >= fecha_inicio) & (df_count_reviews['date'] <= fecha_fin)]
    return [len(df_rango['user_id'].unique()) ,df_rango['recommend'].mean()*100]

In [14]:
df_rango=countreviews('2012-07-15','2013-07-15')
df_rango

[1636, 94.55570530098831]

# GENRE

#Creacion del df para las funciones

In [151]:
df_games.head(2)

,developer,app_name,release_date,id,genres,price
0,kotoshiro,lost summoner kitty,2018-01-04,761140,"['action', 'casual', 'indie', 'simulation', 's...",4.99
1,secret level srl,ironbound,2018-01-04,643980,"['free to play', 'indie', 'rpg', 'strategy']",0.00


In [ ]:
id_to_genre = df_games.dropna(subset='genres').set_index('id')['genres'].to_dict()


In [153]:
def get_all_genres(id_list):
    return id_to_genre.get(id_list,[])

In [154]:
df_items['game_genre']=df_items['item_id'].apply(get_all_genres)

In [155]:
df_items.sample(2)

,user_id,user_url,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,game_genre
1353533,Osiris5622,http://steamcommunity.com/id/Osiris5622,95,76561198065040732,243450,urban trial freestyle,36,0,"['action', 'racing', 'simulation', 'sports']"
4202578,princemart,http://steamcommunity.com/id/princemart,230,76561198040772956,211440,adventures of shuggy,4,0,['indie']


In [156]:
user_genre_playtime=defaultdict(int)
for index, row in df_items.iterrows():
    user=row['user_id']
    game=row['item_id']
    playtime=row['playtime_forever']
    genres =eval(id_to_genre.get(game, '[]'))  # Obtener los géneros del juego
    for genre in genres:
        user_genre = (user, genre)
        user_genre_playtime[user_genre] += playtime
    

In [157]:
user_genre_df = pd.DataFrame(user_genre_playtime.items(), columns=['user_genre', 'playtime'])
user_genre_df.head(10) 

,user_genre,playtime
0,"(76561197970982479, action)",124725
1,"(76561197970982479, strategy)",42223
2,"(76561197970982479, rpg)",39685
3,"(76561197970982479, indie)",19227
4,"(76561197970982479, casual)",2720
5,"(76561197970982479, simulation)",21010
6,"(76561197970982479, adventure)",18462
7,"(76561197970982479, racing)",0
8,"(76561197970982479, sports)",958
9,"(76561197970982479, early access)",469


In [158]:
user_genre_df[['user_id', 'game_genre']] = pd.DataFrame(user_genre_df['user_genre'].tolist(), index=user_genre_df.index)

df_user_genre=user_genre_df.drop(columns='user_genre')
df_user_genre.head(10)

,playtime,user_id,game_genre
0,124725,76561197970982479,action
1,42223,76561197970982479,strategy
2,39685,76561197970982479,rpg
3,19227,76561197970982479,indie
4,2720,76561197970982479,casual
5,21010,76561197970982479,simulation
6,18462,76561197970982479,adventure
7,0,76561197970982479,racing
8,958,76561197970982479,sports
9,469,76561197970982479,early access


In [159]:
df_user_genre = df_user_genre.merge(df_items[['user_id','user_url']], on='user_id', how='left')
df_user_genre.sample(n=2)

,playtime,user_id,game_genre,user_url
17734962,8344,Omega9001,free to play,http://steamcommunity.com/id/Omega9001
16231273,64,MatthewW7,sports,http://steamcommunity.com/id/MatthewW7


In [160]:
df_user_genre=df_user_genre.drop_duplicates()

In [161]:
df_user_genre.to_csv('users_playtime_genre.csv', index=False)

# creacion de funcion genre

In [3]:
df_user_genre=pd.read_csv('users_playtime_genre.csv')

In [10]:
def genre(genero:str):
    df=df_user_genre.groupby('game_genre')['playtime'].sum().reset_index()
    df=df.sort_values('playtime', ascending=False).reset_index().drop(columns='index')
    position = df[df['game_genre'] == genero].index[0] + 1 if genero in df['game_genre'].values else None
    position=str(position)
    return position

5

# creacion de la funcion userforgenre

In [165]:
def userforgenre( genero : str ):
    df = df_user_genre.loc[df_user_genre['game_genre'] == f'{genero}'].sort_values('playtime', ascending=False).reset_index().drop(columns='index')
    return df[['user_id','user_url', 'playtime']].head(5)
    

,user_id,user_url,playtime
0,jimmynoe,http://steamcommunity.com/id/jimmynoe,1062130
1,Evilutional,http://steamcommunity.com/id/Evilutional,684723
2,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,676540
3,tsunamitad,http://steamcommunity.com/id/tsunamitad,661309
4,wolop,http://steamcommunity.com/id/wolop,642773


# DEVELOPER csv

In [168]:
df_developer=df_games[['developer','app_name','price','release_date']].dropna(subset='release_date')
df_developer.sample(10)

,developer,app_name,price,release_date
7273,pinokl games,party hard - dark castle,0.00,2016-09-16
11876,boogygames studios,frontiers.io,0.99,2017-10-12
18375,"smiteworks usa, llc",fantasy grounds - fantastic far east (token pack),4.99,2017-04-03
7362,gritfish,killing time at lightspeed: enhanced edition o...,3.99,2016-09-27
12474,foreverloops gmbh,foreverloops,29.99,2017-11-27
5098,zinertek,fsx: steam edition - natural tree environment ...,11.99,2016-01-12
23288,michael howard,origin of destiny - donation #1,4.99,2016-05-05
20010,sharkbyte studio,ant-gravity: tiny's adventure,12.99,2016-12-15
18003,ronimo games,ted mcpain - awesomenauts character,4.99,2017-04-26
11842,"smiteworks usa, llc",fantasy grounds - pathfinder rpg - hell's rebe...,24.99,2017-10-10


In [169]:
df_developer['year']=df_developer['release_date'].str.extract(r'(\d{4})')
df_developer=df_developer.dropna(subset='year').drop(columns='release_date')

In [170]:
df_developer.sample(3)

,developer,app_name,price,year
1692,"jackbox games, inc.",you don't know jack television,2.99,1997
7598,blind bird,stay close,8.99,2016
28341,sarepta studio,shadow puppeteer,14.99,2014


In [172]:
df_developer.to_csv('df_developer.csv', index=False)

In [350]:
df_reviews.sample(10)

,user_id,user_url,item_id,recommend,review,date
49520,76561198057354755,http://steamcommunity.com/profiles/76561198057...,4000,1,1,2015-10-26
54147,LukeIsFukinAwesome,http://steamcommunity.com/id/LukeIsFukinAwesome,72850,1,2,2013-03-13
31360,HaZeFaMe,http://steamcommunity.com/id/HaZeFaMe,367120,1,1,2015-11-26
16691,Pdhelliot,http://steamcommunity.com/id/Pdhelliot,212680,1,0,2014-04-24
10978,digirice,http://steamcommunity.com/id/digirice,221260,1,1,2015-06-27
18309,akamexxswag,http://steamcommunity.com/id/akamexxswag,105600,1,2,2015-06-28
53905,76561198078294706,http://steamcommunity.com/profiles/76561198078...,104900,1,1,2014-10-02
21776,frankiesayschillax,http://steamcommunity.com/id/frankiesayschillax,208650,0,0,2015-06-24
23320,Brownies_less_than_3,http://steamcommunity.com/id/Brownies_less_than_3,570,1,2,2014-12-08
57675,SicilianSorcerer,http://steamcommunity.com/id/SicilianSorcerer,203160,1,0,2015-07-23


In [173]:
len(df_developer)

29966

# creacion de funcion developer

In [33]:
df_developer=pd.read_csv('df_developer.csv')

In [35]:
def developer(desarrollador: str):
    desarrollador=desarrollador
    desarrollador_lower = desarrollador.lower()
    df = df_developer.loc[df_developer['developer'] == f'{desarrollador_lower}'].sort_values('year')
    df=df.drop_duplicates()
    yearB = 0
    dict_años = defaultdict(list)  # Changed the dictionary value type to list
    items = 0
    items_free=0
    for index, row in df.iterrows():
        year = row['year']
        price = row['price']
        if  year != yearB:
            items = 0  # Restablece la cuenta de elementos si es un nuevo año
            yearB = year
            items_free=0
              
        if price == 0.00:
            items_free+=1
            dict_años[year] = [f'{items_free}', items] 
        else:
            items += 1
            dict_años[year] = [f'{items_free}', items]
    texto=' Free content %'

    df = pd.DataFrame(dict_años.values(), dict_años.keys()).rename(columns={0:f'{desarrollador}{texto}', 1: 'lanzamientos ese año'})
    df[f'{desarrollador}{texto}']= round((pd.to_numeric(df[f'{desarrollador}{texto}']) / df['lanzamientos ese año']*100), 2)
    return df

In [36]:
xd=developer('capcom')


In [37]:
xd

,capcom Free content %,lanzamientos ese año
2008,0.00,2
2009,0.00,3
2011,0.00,8
2013,0.00,14
2014,0.00,21
2015,0.00,33
2016,9.09,11
2017,2.56,39


# SENTIMENT csv

In [356]:
df_games

,developer,app_name,release_date,id,genres,price
0,kotoshiro,lost summoner kitty,2018-01-04,761140,"['action', 'casual', 'indie', 'simulation', 's...",4.99
1,secret level srl,ironbound,2018-01-04,643980,"['free to play', 'indie', 'rpg', 'strategy']",0.00
2,poolians.com,real pool 3d - poolians,2017-07-24,670290,"['casual', 'free to play', 'indie', 'simulatio...",0.00
3,彼岸领域,弹炸人2222,2017-12-07,767400,"['action', 'adventure', 'casual']",0.99
4,NaN,log challenge,NaN,773570,[],2.99
...,...,...,...,...,...,...
32128,"nikita ""ghost_rus""",colony on mars,2018-01-04,773640,"['casual', 'indie', 'simulation', 'strategy']",1.99
32129,sacada,logistical: south africa,2018-01-04,733530,"['casual', 'indie', 'strategy']",4.99
32130,laush dmitriy sergeevich,russian roads,2018-01-04,610660,"['indie', 'racing', 'simulation']",1.99
32131,"xropi,stev3ns",exit 2 - directions,2017-09-02,658870,"['casual', 'indie']",4.99


In [352]:
df_sentiment_analysis=df_reviews.copy()
df_sentiment_analysis=df_sentiment_analysis[['item_id','review']]
df_sentiment_analysis['review']=df_sentiment_analysis['review'].fillna(1)

In [354]:
df_sentiment_analysis=df_sentiment_analysis.dropna(subset='item_id')

In [355]:
df_sentiment_analysis

,item_id,review
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,0
...,...,...
59300,70,2
59301,362890,2
59302,273110,2
59303,730,2


In [357]:
id_to_release = df_games.dropna(subset='id').set_index('id')['release_date'].to_dict()

In [358]:
def id_release(id):
    return id_to_release.get(id,np.nan)

In [359]:
df_sentiment_analysis['año_lanzamiento']=df_sentiment_analysis['item_id'].apply(id_release)

In [360]:
df_sentiment_analysis['año_lanzamiento']=df_sentiment_analysis['año_lanzamiento'].str.extract(r'(\d{4})')

In [361]:
df_sentiment_analysis

,item_id,review,año_lanzamiento
0,1250,2,2009
1,22200,2,2009
2,43110,2,NaN
3,251610,2,NaN
4,227300,0,2013
...,...,...,...
59300,70,2,1998
59301,362890,2,2015
59302,273110,2,2014
59303,730,2,2012


In [362]:
df_sentiment_analysis=df_sentiment_analysis.groupby('año_lanzamiento')['review'].apply(list).reset_index()

In [386]:
df_sentiment_analysis.head()

,año_lanzamiento,review
0,1989,[2]
1,1990,"[0, 2, 2, 2, 2]"
2,1991,[2]
3,1992,"[2, 2, 2]"
4,1993,"[2, 0, 0, 0, 2]"


In [364]:
df_sentiment_analysis.to_csv('df_sentiment_analysis.csv', index=False)

# creacion funcion sentiment_analysis

In [38]:
sentiment=pd.read_csv('df_sentiment_analysis.csv')

In [40]:
def sentiment_analysis(año: int):
    df = sentiment.copy()
    filtered_df = df[df['año_lanzamiento'] == año]['review']
    
    if not filtered_df.empty:
        positivo = filtered_df.iloc[0].count('2')
        negativo = filtered_df.iloc[0].count('0')
        neutro = filtered_df.iloc[0].count('1')
        return f'Negative = {negativo}, Neutral = {neutro}, Positive = {positivo}'
    else:
        return 'No hay reseñas para este año de lanzamiento'

In [41]:
xd=sentiment_analysis(2016)
xd

'Negative = 694, Neutral = 545, Positive = 1759'

In [196]:
positivo= df.count(2)
positivo

44